In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv("/kaggle/input/laptop-price/laptop_price.csv", encoding= 'unicode_escape')
data.head()

The dataset used for this notebook is the "Laptop Price" dataset on Kaggle, 
which can be found at the following link 
https://www.kaggle.com/muhammetvarl/laptop-pricehttps://www.kaggle.com/muhammetvarl/laptop-price.

At the above link, there is also a description of the variables, which will be repeated below.

1 Company- String -Laptop Manufacturer

2 Product -String -Brand and Model

3 TypeName -String -Type (Notebook, Ultrabook, Gaming, etc.)

4 Inches -Numeric- Screen Size

5 ScreenResolution -String- Screen Resolution

6 Cpu- String -Central Processing Unit (CPU)

7 Ram -String- Laptop RAM

8 Memory -String- Hard Disk / SSD Memory

9 GPU -String- Graphics Processing Units (GPU)

10 OpSys -String- Operating System

11 Weight -String- Laptop Weight

12 Price_euros -Numeric- Price (Euro)

## **Goal**:  Predict the price of a laptop from the other variables.

# Hypothesis Generation

Here we will try to generate some hypotheses about what could affect the price of a laptop.

1. Laptops made by well-known companies like Apple and Microsoft might be more expensive.

2. Laptops with a bigger screen size might be more expensive.

3. Laptops with a high screen resolution might be more expensive.

4. Laptops with a high CPU might be more expensive.

5. Laptops with a higher RAM might be more expensive.

6. Laptops with a large memory might be more expensive.

7. Laptops with a high GPU might be more expensive.

8. Laptops without an operating sysytem might be cheaper.

9. Lighter Laptops might be more expensive than heavier laptops
   because they could be newer with more advanced technology.

In [43]:
# Here we will important the required packages.

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Preliminary Analysis

In [4]:
data.shape

In [5]:
data.dtypes

In [6]:
# First, we will check if there are any duplicate laptop Id's.

idsUnique = len(set(data['laptop_ID']))
idsTotal = data.shape[0]
idsdupe = idsTotal - idsUnique
print(idsdupe)

In [7]:
# Since there are no duplicates, we will drop the laptop_ID column.

data.drop(['laptop_ID'], axis=1, inplace=True)

In [8]:
data.shape

In [9]:
data.head()


 # Step 2: Identify the features and target variable.

In [10]:
y = data["Price_euros"]
X = data.drop("Price_euros", axis=1)

In [11]:
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

# Step 3: Split the data into training and test sets.

In [12]:
# We will put 80% of the data in the training set and the remaining 20% in the test set.

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8,random_state=0)


In [13]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

In [14]:
X_train.head()

In [15]:
X_test.head()

In [16]:
y_train.head()

In [17]:
y_test.head()

# Step 4: Data Exploration

We will perform data exploration on the training set.

In [18]:
X_train.head(10)

In [19]:
X_train.tail(10)

In [20]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)


First, we look at some descriptive statistics 
and the distribution plot for the target variable. 

In [42]:
y_train.describe()

In [49]:
f, ax = plt.subplots(figsize=(10, 8))
fig = sns.distplot(y_train)

Now, we will look at the features.

In [21]:
X_train.dtypes

In [22]:
X_train.info()

In [33]:
X_train.describe()

In [25]:
# Let us look at some descriptive statistics for the remaining categorical variables.

X_train.describe(include=[np.object])

Now, we will look at each column individually.

In [28]:
X_train['Company'].value_counts()

In [29]:
X_train['Product'].value_counts()

In [30]:
X_train['TypeName'].value_counts()

In [31]:
X_train['Inches'].value_counts()

In [35]:
X_train['ScreenResolution'].value_counts()

In [36]:
X_train['Cpu'].value_counts()

In [37]:
X_train['Ram'].value_counts()

In [38]:
X_train['Memory'].value_counts()

In [39]:
X_train['Gpu'].value_counts()

In [40]:
X_train['OpSys'].value_counts()

In [41]:
X_train['Weight'].value_counts()

In [54]:
train, test = train_test_split(data, train_size=0.8,random_state=0)

In [55]:
train.head()

I made an error.  
I need to separate the target variable from the features 
later on in the data processing phase.
I will fix this in the next version.